**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("C:\\Users\\saklo\\Downloads\\HW3\\ffmpeg-4.1-win64-static\\ffmpeg-4.1-win64-static\\bin\\")
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

C:\Users\saklo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The epsilon parameter allows us to control the exploration : $\epsilon$-greedy strategy. If $\epsilon$= 0, then we will act based on what we have learne so far.This choice is based on all the previous cumulated knowledge. If $\epsilon$ =1, an action will be chosen randomly at each time step t . 

In order to not make the actions being too biased by the action and states history, it is essential to add an exploration criteria.

For example, consider a grid world where the agent’s current Q-estimates are right everywhere except one state where it tells it to walk into a wall that returns back to the same state. If the agent reaches this state, then it will forever be stuck and your overall performance will appear terrible, even though everywhere else in the world learning converged. By keeping an exploration policy like epsilon greedy, you allow the agent to randomly fall out of these kinds of slight misestimation loops and not have them dominate the measure of overall performance.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array contains the information about the cases where the rat can go. There is -1 in the boundaries where it can not go, 0 where it can actually go and finally 1 in its current position.
The board represent the reward of each cell of the grid. The value board[x,y] is the corresponding reward of the position (x,y) occupied by the rat. In the cell with cheese the reward is (+0.5) and in poisonous cells (-1) and 0 otherwise. The board is updated everytime a piece of cheese is eaten (0 instead of 0.5).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        
        return  np.random.randint(0, self.n_action, size=1)[0]


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.5/17.0. Average score (-2.5)
Win/lose count 10.0/19.0. Average score (-5.75)
Win/lose count 7.5/8.0. Average score (-4.0)
Win/lose count 11.5/14.0. Average score (-3.625)
Win/lose count 9.5/15.0. Average score (-4.0)
Win/lose count 10.0/15.0. Average score (-4.166666666666667)
Win/lose count 14.0/15.0. Average score (-3.7142857142857144)
Win/lose count 7.5/10.0. Average score (-3.5625)
Win/lose count 13.5/12.0. Average score (-3.0)
Win/lose count 9.0/14.0. Average score (-3.2)
Win/lose count 12.0/17.0. Average score (-3.3636363636363638)
Win/lose count 8.0/11.0. Average score (-3.3333333333333335)
Win/lose count 10.0/15.0. Average score (-3.4615384615384617)
Win/lose count 12.5/23.0. Average score (-3.9642857142857144)
Win/lose count 9.5/18.0. Average score (-4.266666666666667)
Final score: -4.266666666666667


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have 

\begin{align}
Q^{\pi}(s,a)&=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& =  r(s,a) +\gamma E[\sum_{k=0}^{\infty} \gamma^k r_{t+k+1}|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] ~Law~of~total ~expectation
 \\
& =  r(s,a) + \gamma \sum_{(s',a')}(p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

Then we have:

\begin{align}
Q^{*}(s,a) &= \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= r(s,a) + \max_{\pi '}\max_{a'}\gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )Q^{\pi'}(s',a')\\ 
\end{align}

We exchange the maximums and the sum :

\begin{align}
Q^{*}(s,a)&= r(s,a) + \gamma  \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'}\max_{\pi'}Q^{\pi'}(s',a') \\
&= r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a') \\ 
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align}

The proposed objective function ensures that the learned Q-value verifies the Bellman equation and that the neural network with parameter $\theta$ will perfectly converge. 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>self.max_memory:
            del self.memory[0]
        
        self.memory.append(m)

    def random_access(self):
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        sreshaped=s.reshape([1,s.shape[0],s.shape[1],s.shape[2]])
        return np.argmax(self.model.predict(sreshaped)[0,:])


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            shape_=[1,s_.shape[0],s_.shape[1],s_.shape[2]]
            target_q[i] =self.model.predict(s_.reshape(shape_))[0]
    
            if game_over_:
                ######## FILL IN

                input_states[i] = s_
                target_q[i,a_] = r_ #we do not update the prediction of the  network
            else:
                ######## FILL IN
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape(shape_))[0])
            
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu')) #simple one hidden layer network (see performance of other methods below)
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [26]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.25, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
test(agent, env, epochs_train, prefix='fc_test')# we test on 50 epochs as for the train phase
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0148 | Win/lose count 5.5/9.0 (-3.5)
Epoch 001/050 | Loss 0.0115 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/050 | Loss 0.0199 | Win/lose count 3.0/2.0 (1.0)
Epoch 003/050 | Loss 0.0206 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/050 | Loss 0.0158 | Win/lose count 3.5/5.0 (-1.5)
Epoch 005/050 | Loss 0.0163 | Win/lose count 6.0/3.0 (3.0)
Epoch 006/050 | Loss 0.1326 | Win/lose count 3.0/1.0 (2.0)
Epoch 007/050 | Loss 0.0134 | Win/lose count 5.5/3.0 (2.5)
Epoch 008/050 | Loss 0.0043 | Win/lose count 6.0/5.0 (1.0)
Epoch 009/050 | Loss 0.0045 | Win/lose count 6.5/5.0 (1.5)
Epoch 010/050 | Loss 0.0040 | Win/lose count 4.5/4.0 (0.5)
Epoch 011/050 | Loss 0.0034 | Win/lose count 6.0/4.0 (2.0)
Epoch 012/050 | Loss 0.0079 | Win/lose count 9.5/1.0 (8.5)
Epoch 013/050 | Loss 0.0022 | Win/lose count 11.5/6.0 (5.5)
Epoch 014/050 | Loss 0.0050 | Win/lose count 8.5/3.0 (5.5)
Epoch 015/050 | Loss 0.0043 | Win/lose count 5.0/6.0 (-1.0)
Epoch 016/050 | Loss 0.0052 | Win/lose count 8.5/1.

 I tried to play with the learning rate,  the epsilon parameter batch size and memory size that seem to play an important role as a whole.

| Learning Rate| Epsilon  | Batch size|Memory size| Final Test Score |
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
|0.1 |0.1 |32 |2000 |3.7|
|**0.1** |**0.25** |**32** |**2000** |**4.1**|
|0.1 |0.35 |32 |2000 |3.35|
|0.1 |0.5 |32 |2000 |2.98|
|0.3 |0.1 |32 |2000 |3.07|
|0.5 |0.1 |32 |2000 |3.27|
|0.7 |0.1 |32 |2000 |3.46|
|0.7 |0.25 |32 |2000  |2.61|
|0.1 |0.1 |16 |2000 |3.31|
|0.1|0.1 |64 |2000 |2.77|
|0.7 |0.25 |16 |2000  |2.37|
|0.1 |0.1 |32 |1000|2.14|
|0.1 |0.1 |32 |3000 |1.79|


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=2, activation='relu', input_shape=(5,5,self.n_state,)))
        model.add(Conv2D(32, kernel_size=2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [32]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
test(agent, env, epochs_train, prefix='cnn_test')# we test on 50 epochs as for the train phase
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0035 | Win/lose count 6.0/5.0 (1.0)
Epoch 001/050 | Loss 0.0037 | Win/lose count 3.0/3.0 (0.0)
Epoch 002/050 | Loss 0.0111 | Win/lose count 1.0/3.0 (-2.0)
Epoch 003/050 | Loss 0.0073 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/050 | Loss 0.0034 | Win/lose count 3.0/0 (3.0)
Epoch 005/050 | Loss 0.0031 | Win/lose count 10.0/6.0 (4.0)
Epoch 006/050 | Loss 0.0022 | Win/lose count 6.5/2.0 (4.5)
Epoch 007/050 | Loss 0.0096 | Win/lose count 4.5/5.0 (-0.5)
Epoch 008/050 | Loss 0.0014 | Win/lose count 2.0/1.0 (1.0)
Epoch 009/050 | Loss 0.0080 | Win/lose count 6.0/2.0 (4.0)
Epoch 010/050 | Loss 0.0606 | Win/lose count 6.5/0 (6.5)
Epoch 011/050 | Loss 0.0013 | Win/lose count 4.5/0 (4.5)
Epoch 012/050 | Loss 0.0015 | Win/lose count 7.0/2.0 (5.0)
Epoch 013/050 | Loss 0.0015 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/050 | Loss 0.0014 | Win/lose count 7.0/2.0 (5.0)
Epoch 015/050 | Loss 0.0016 | Win/lose count 9.0/3.0 (6.0)
Epoch 016/050 | Loss 0.0029 | Win/lose count 8.0/0 (8.0)
Ep

In [28]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
test(agent, env, epochs_train, prefix='cnn_test')# we test on 50 epochs as for the train phase
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0132 | Win/lose count 5.5/7.0 (-1.5)
Epoch 001/050 | Loss 0.0081 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/050 | Loss 0.0090 | Win/lose count 4.5/5.0 (-0.5)
Epoch 003/050 | Loss 0.0068 | Win/lose count 4.5/6.0 (-1.5)
Epoch 004/050 | Loss 0.0024 | Win/lose count 6.5/2.0 (4.5)
Epoch 005/050 | Loss 0.0061 | Win/lose count 7.0/3.0 (4.0)
Epoch 006/050 | Loss 0.0622 | Win/lose count 7.5/2.0 (5.5)
Epoch 007/050 | Loss 0.0027 | Win/lose count 6.0/1.0 (5.0)
Epoch 008/050 | Loss 0.0014 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/050 | Loss 0.0055 | Win/lose count 9.5/3.0 (6.5)
Epoch 010/050 | Loss 0.0019 | Win/lose count 6.0/2.0 (4.0)
Epoch 011/050 | Loss 0.0027 | Win/lose count 2.5/1.0 (1.5)
Epoch 012/050 | Loss 0.0031 | Win/lose count 10.5/2.0 (8.5)
Epoch 013/050 | Loss 0.0012 | Win/lose count 11.5/2.0 (9.5)
Epoch 014/050 | Loss 0.0018 | Win/lose count 11.0/2.0 (9.0)
Epoch 015/050 | Loss 0.0533 | Win/lose count 14.5/2.0 (12.5)
Epoch 016/050 | Loss 0.0028 | Win/lose count 1.5

* Architecture 1: 2 Convolution Layer ((64,(3,3)),((32,(3,3)) , relu activations
* Architecture 2: 2 Convolution Layer ((64,(2,2) and then (32,(2,2)), relu activations
* Architecture 3: 2 Convolution Layer ((32,(2,2),(64,(3,3))then a MaxPooling layer (2,2), two dense layers, relu activations with dropout =0.25


| Architecture       |     Epochs Train    |        Epsilon | Learning Rate | Batch size|Memory size| Final Test Score|
| :------------ | :-------------: | -------------: | -------------: |-------------: |-------------: |-------------: |
| 1       |      50     |         0.1 |0.1 |32 |2000  | 2.24|
|** 2 **  | ** 50**      |         **0.1** |**0.1** |**32** |**2000**  | **6.09** |
| 2       |      50      |         0.1 |0.7 |32 |2000  | 4.51 |
| 3       |      50      |         0.1 |0.7 |32 |2000  |3.66|
| 3       |      50      |         0.1 |0.1 |32 |2000  |1.11|
|   2   |     50      |         0.1 |0.25 |32 |2000  | 5.49|                                              
                                              

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [45]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.25, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 14.0/0. Average score (14.0)
Win/lose count 11.5/0. Average score (12.75)
Win/lose count 51.5/0. Average score (25.666666666666668)
Win/lose count 10.0/0. Average score (21.75)
Win/lose count 9.5/0. Average score (19.3)
Win/lose count 10.5/0. Average score (17.833333333333332)
Win/lose count 5.5/0. Average score (16.071428571428573)
Win/lose count 8.5/0. Average score (15.125)
Win/lose count 17.0/1.0. Average score (15.222222222222221)
Win/lose count 9.5/0. Average score (14.65)
Win/lose count 14.5/0. Average score (14.636363636363637)
Win/lose count 5.5/0. Average score (13.875)
Win/lose count 14.0/0. Average score (13.884615384615385)
Win/lose count 8.5/0. Average score (13.5)
Win/lose count 6.0/0. Average score (13.0)
Final score: 13.0
Test of the FC
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 4.5/0. Average score (2.1666666666666665)
Win/lose count 0.5/1.0. Average score (1.5)
Win/lose count 1.0

In [46]:
HTML(display_videos('cnn_test10.mp4'))

In [47]:
HTML(display_videos('fc_test10.mp4'))



Below we found the results for different values of the temperature parameter for the two designed networks. 

|    Temperature    |   Final Test Score FC|Final Test Score CNN|
| :------------ | :-------------: | -------------: | 
| ** 0.1   **     | -0.1    |     1.1  |
| ** 0.2   **      |   -0.33      | 2.4 |
| ** 0.3   **     |  -0.3   |   4.6  |
| ** 0.4   **        |-0.46  |5.8|
| ** 0.5   **      |  -1.3   |  6.63 |
| ** 0.6   **      |    0.1   | 8.26 |
| ** 0.7   **      |  0.83  |   9.43 |
| ** 0.8   **     |    2.43     | 15   |
| ** 0.9   **     |  1.43    |  13  |

We can see that the performance of the CNN is widely better than the one of fully connected networks. the performance of the CNN is increasing with the temperature while the one of the FC stagnates around 0. We notice from the videos that  the rat is stuck in some positions without getting to other unexplored ones. Hence, the algorithms tend to not explore the map.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [12]:
def train_explore(agent,env,epoch,prefix=''):
    
    decay_parameter_epsilon=0.1
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        #update epsilon
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) # maluses when getting back to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        #initialize to zero
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state





In [51]:
# Training
#we modify the initialization of epsilon
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0147 | Win/lose count 8.0/29.400000000000045 (-21.400000000000045)
Epoch 001/050 | Loss 0.0081 | Win/lose count 8.5/27.00000000000006 (-18.50000000000006)
Epoch 002/050 | Loss 0.0116 | Win/lose count 6.0/28.700000000000085 (-22.700000000000085)
Epoch 003/050 | Loss 0.0305 | Win/lose count 19.0/22.599999999999984 (-3.5999999999999837)
Epoch 004/050 | Loss 0.0080 | Win/lose count 10.5/20.100000000000026 (-9.600000000000026)
Epoch 005/050 | Loss 0.0056 | Win/lose count 22.0/18.6 (3.3999999999999986)
Epoch 006/050 | Loss 0.0049 | Win/lose count 18.0/18.39999999999999 (-0.3999999999999915)
Epoch 007/050 | Loss 0.0065 | Win/lose count 14.5/21.500000000000057 (-7.000000000000057)
Epoch 008/050 | Loss 0.0038 | Win/lose count 13.5/20.099999999999994 (-6.599999999999994)
Epoch 009/050 | Loss 0.0061 | Win/lose count 18.5/18.7 (-0.1999999999999993)
Epoch 010/050 | Loss 0.0052 | Win/lose count 17.5/20.50000000000001 (-3.0000000000000107)
Epoch 011/050 | Loss 0.0140 | Win/lose

In [52]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 22.5/1.0. Average score (21.5)
Win/lose count 18.0/2.0. Average score (18.75)
Win/lose count 19.5/1.0. Average score (18.666666666666668)
Win/lose count 25.0/2.0. Average score (19.75)
Win/lose count 28.0/3.0. Average score (20.8)
Win/lose count 23.0/2.0. Average score (20.833333333333332)
Win/lose count 23.0/0. Average score (21.142857142857142)
Win/lose count 22.0/0. Average score (21.25)
Win/lose count 4.0/0. Average score (19.333333333333332)
Win/lose count 19.5/1.0. Average score (19.25)
Win/lose count 26.5/1.0. Average score (19.818181818181817)
Win/lose count 15.5/2.0. Average score (19.291666666666668)
Win/lose count 28.0/1.0. Average score (19.884615384615383)
Win/lose count 29.0/2.0. Average score (20.392857142857142)
Win/lose count 24.0/1.0. Average score (20.566666666666666)
Final score: 20.566666666666666


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [31]:
class DQN_mimicking(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_mimicking, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        sreshaped=s.reshape([1,s.shape[0],s.shape[1],s.shape[2]])
        return np.argmax(self.model.predict(sreshaped)[0,:])


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        #no need to the other parameters, we do not use anymore the bellman equation, we just mimick
        # Two steps: first memorize the states, second learn from the pool
        
        #mimick what the expert has done
        self.memory.remember([s_[:,:,1:], a_]) 

        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 5  #maximizing target function
            
        l = self.model.train_on_batch(input_states, target_q)


        return l
       
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_CNN_mimicking(DQN_mimicking):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_CNN_mimicking, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=2, activation='relu', input_shape=(5,5,self.n_state,)))
        model.add(Conv2D(32, kernel_size=2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [32]:
def train_mimicking(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [33]:
epochs_train = 50
expert = DQN_CNN(size, lr=.1, epsilon =0.7, memory_size=2000, batch_size = 32,n_state=3)
expert.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
agent_mimick = DQN_CNN_mimicking(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimicking(expert,agent_mimick, env, epochs_train, prefix='cnn_train_mimicking')

Epoch 000/050 | Loss 0.1805 | Win/lose count 7.0/0 (7.0)
Epoch 001/050 | Loss 1.3311 | Win/lose count 21.0/2.0 (19.0)
Epoch 002/050 | Loss 1.8341 | Win/lose count 27.5/0 (27.5)
Epoch 003/050 | Loss 0.7866 | Win/lose count 0/0 (0)
Epoch 004/050 | Loss 0.8401 | Win/lose count 27.5/1.0 (26.5)
Epoch 005/050 | Loss 1.0054 | Win/lose count 18.5/2.0 (16.5)
Epoch 006/050 | Loss 0.8517 | Win/lose count 21.0/4.0 (17.0)
Epoch 007/050 | Loss 1.2436 | Win/lose count 24.0/2.0 (22.0)
Epoch 008/050 | Loss 0.8888 | Win/lose count 18.5/0 (18.5)
Epoch 009/050 | Loss 1.0442 | Win/lose count 21.0/1.0 (20.0)
Epoch 010/050 | Loss 0.7284 | Win/lose count 9.0/1.0 (8.0)
Epoch 011/050 | Loss 0.9581 | Win/lose count 21.5/0 (21.5)
Epoch 012/050 | Loss 0.9751 | Win/lose count 18.5/2.0 (16.5)
Epoch 013/050 | Loss 1.0026 | Win/lose count 15.5/0 (15.5)
Epoch 014/050 | Loss 1.1674 | Win/lose count 20.5/2.0 (18.5)
Epoch 015/050 | Loss 1.2009 | Win/lose count 22.5/1.0 (21.5)
Epoch 016/050 | Loss 1.0826 | Win/lose count 2

In [34]:
env_test =Environment(grid_size=size, max_time=T, temperature=0.3)
test(agent_mimick,env_test,epochs_test,prefix='cnn_test_mimicking')

Win/lose count 5.5/1.0. Average score (4.5)
Win/lose count 14.5/1.0. Average score (9.0)
Win/lose count 5.0/0. Average score (7.666666666666667)
Win/lose count 14.0/1.0. Average score (9.0)
Win/lose count 15.0/1.0. Average score (10.0)
Win/lose count 7.0/0. Average score (9.5)
Win/lose count 5.5/0. Average score (8.928571428571429)
Win/lose count 11.0/0. Average score (9.1875)
Win/lose count 1.0/0. Average score (8.277777777777779)
Win/lose count 12.5/1.0. Average score (8.6)
Win/lose count 2.5/0. Average score (8.045454545454545)
Win/lose count 7.5/0. Average score (8.0)
Win/lose count 14.5/1.0. Average score (8.423076923076923)
Win/lose count 8.5/0. Average score (8.428571428571429)
Win/lose count 2.5/0. Average score (8.033333333333333)
Final score: 8.033333333333333


In [36]:
HTML(display_videos('cnn_test_mimicking10.mp4'))

We can see that the mimicking agent performs not so well, it suffers from the same issue as the unexploring agent. It van easily get stuck.
The score is of course lower than the score of the exploring agent. It is expected that mimicking or copying the expert would be worse than exploring and learning on its own.

***